In [0]:
%pip install delta-sharing

In [0]:
# TODO # Read in Ray Data from Delta Sharing for memory efficient load compared to from_spark (i.e. no need to do in-memory Spark -> Ray Data)
profile_file = "config.share"
SHARE_NAME = 'internal-ray-share'
SHARE_SCHEMA = 'ray_gtm_examples'
SHARE_VOLUME = 'prophet_binaries'
table_url = f"{profile_file}#{SHARE_NAME}.{SHARE_SCHEMA}.{SHARE_VOLUME}"


In [0]:
from delta_sharing import SharingClient

client = SharingClient(profile_file)

df = spark.read.format("delta").load(f"delta.{SHARE_NAME}.{SHARE_SCHEMA}.{SHARE_VOLUME}")

# Display the data
df.show()

# # List all files in the specific share volume
# files = client.list_files_in_table(SHARE_NAME, SHARE_SCHEMA, SHARE_VOLUME)
# display(files)

In [0]:
experiment_name = '/Users/jon.cheung@databricks.com/ray_prophet_map_batches'

import pickle
from pprint import pprint
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.catalog import *


catalog            = 'main'
schema             = 'ray_gtm_examples'
volume             = 'prophet_binaries'
volume_path        = f"/Volumes/{catalog}/{schema}/{volume}" 


class ProphetInferenceRouter(mlflow.pyfunc.PythonModel):
  def __init__(self, volume_path):
    self.workspace_model_binaries = volume_path

  def load_context(self, context): 
    self.router = pd.read_parquet(context['model_router'])
    
  def _connect_to_workspace_client():
    w = WorkspaceClient(host='https://e2-demo-field-eng.cloud.databricks.com/',
                        token='xxx')


  def predict(self, model_input):
    
    model_path = self.router.loc[self.router['group_name'] == model_input['identifier'], 'model_path'].values[0]
    model = pickle.loads(model_path)
    future = model.make_future_dataframe(periods=horizon)
    forecast = model.predict(future)

    
  

In [0]:
{
  "name": "endpoint-name",
  "config": {
    "served_entities": [
      {
        "entity_name": "model-name",
        "entity_version": "1",
        "workload_size": "Small",
        "scale_to_zero_enabled": "true",
        "environment_vars": {
          "OPENAI_API_KEY": "{{secrets/my_secret_scope/my_secret_key}}"
        }
      }
    ]
  }
}